In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
import random

Using TensorFlow backend.
d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\alex\code\machine-learning\env36\lib\site-pa

In [2]:
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)

In [3]:
df_orig = pd.read_csv("train.csv")
print(df_orig.shape)
print("Null Counts:")
print(df_orig.isnull().sum()[df_orig.isnull().sum() > 0])

(1460, 81)
Null Counts:
LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [4]:
df = df_orig.drop(columns=["MiscFeature", "Fence", "PoolQC", "Alley", "Id", "SalePrice"])
print(df.shape)

(1460, 75)


In [5]:
def fill_nulls(df, mean, mode):
    df.fillna(mean, inplace=True)
    df.fillna(mode, inplace=True)

mean = df.mean()
mode = df.mode().iloc[0]
fill_nulls(df, mean, mode)
print(df.isnull().sum()[df.isnull().sum() > 0])

Series([], dtype: int64)


In [6]:
def encode_categorical_features(df, enc):
    # Flatten enc.categories_ which is a list of np.array
    cat_list = np.concatenate(enc.categories_).ravel()
    df[cat_list] = pd.DataFrame(enc.transform(df[categorical_features]).toarray(), index=df.index)
    df = df.drop(columns=categorical_features)
    return df

categorical_features = df.select_dtypes(exclude=[np.number]).columns
onehot = OneHotEncoder(handle_unknown='ignore')
onehot.fit(df[categorical_features])
df = encode_categorical_features(df, onehot)
print(df.shape)

(1460, 203)


In [7]:
def standardize_numerical_features(df):
    numerical_features = df.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    return df

df = standardize_numerical_features(df)

In [8]:
X = df
y = df_orig.SalePrice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1095, 203) (1095,)
(365, 203) (365,)


In [58]:
model = Sequential()
model.add(Dense(units=90, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=90, activation='relu', input_dim=X_train.shape[1]))
#model.add(Dropout(0.2))
#model.add(Dense(units=20, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(units=1, activation=None))

model.compile(loss='mean_squared_logarithmic_error',
              optimizer='Adam',
              metrics=['mean_squared_logarithmic_error'])

model.fit(X_train, y_train, epochs=1000, batch_size=16)
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)
print(train_loss[0], test_loss[0])

Epoch 1/1000
1095/1095 [==============================] - 2s 2ms/step - loss: 94.5918 - mean_squared_logarithmic_error: 94.5918
Epoch 2/1000
1095/1095 [==============================] - 0s 90us/step - loss: 65.3497 - mean_squared_logarithmic_error: 65.3497
Epoch 3/1000
1095/1095 [==============================] - 0s 90us/step - loss: 52.2722 - mean_squared_logarithmic_error: 52.2722
Epoch 4/1000
1095/1095 [==============================] - 0s 95us/step - loss: 44.1038 - mean_squared_logarithmic_error: 44.1038
Epoch 5/1000
1095/1095 [==============================] - 0s 90us/step - loss: 38.3269 - mean_squared_logarithmic_error: 38.3269
Epoch 6/1000
1095/1095 [==============================] - 0s 90us/step - loss: 33.9568 - mean_squared_logarithmic_error: 33.9568
Epoch 7/1000
1095/1095 [==============================] - 0s 90us/step - loss: 30.4956 - mean_squared_logarithmic_error: 30.4956
Epoch 8/1000
1095/1095 [==============================] - 0s 90us/step - loss: 27.6696 - mean_squa

1095/1095 [==============================] - 0s 90us/step - loss: 1.3987 - mean_squared_logarithmic_error: 1.3987
Epoch 65/1000
1095/1095 [==============================] - 0s 84us/step - loss: 1.3275 - mean_squared_logarithmic_error: 1.3275
Epoch 66/1000
1095/1095 [==============================] - 0s 83us/step - loss: 1.2593 - mean_squared_logarithmic_error: 1.2593
Epoch 67/1000
1095/1095 [==============================] - 0s 84us/step - loss: 1.1943 - mean_squared_logarithmic_error: 1.1943
Epoch 68/1000
1095/1095 [==============================] - 0s 83us/step - loss: 1.1321 - mean_squared_logarithmic_error: 1.1321
Epoch 69/1000
1095/1095 [==============================] - 0s 84us/step - loss: 1.0728 - mean_squared_logarithmic_error: 1.0728
Epoch 70/1000
1095/1095 [==============================] - 0s 84us/step - loss: 1.0161 - mean_squared_logarithmic_error: 1.0161
Epoch 71/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.9621 - mean_squared_logarithmic_erro

1095/1095 [==============================] - 0s 86us/step - loss: 0.0603 - mean_squared_logarithmic_error: 0.0603
Epoch 128/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0588 - mean_squared_logarithmic_error: 0.0588
Epoch 129/1000
1095/1095 [==============================] - 0s 86us/step - loss: 0.0574 - mean_squared_logarithmic_error: 0.0574
Epoch 130/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0561 - mean_squared_logarithmic_error: 0.0561
Epoch 131/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0548 - mean_squared_logarithmic_error: 0.0548
Epoch 132/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0536 - mean_squared_logarithmic_error: 0.0536
Epoch 133/1000
1095/1095 [==============================] - 0s 83us/step - loss: 0.0525 - mean_squared_logarithmic_error: 0.0525
Epoch 134/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0514 - mean_squared_logarithm

1095/1095 [==============================] - 0s 85us/step - loss: 0.0208 - mean_squared_logarithmic_error: 0.0208
Epoch 191/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0206 - mean_squared_logarithmic_error: 0.0206
Epoch 192/1000
1095/1095 [==============================] - 0s 86us/step - loss: 0.0203 - mean_squared_logarithmic_error: 0.0203
Epoch 193/1000
1095/1095 [==============================] - 0s 86us/step - loss: 0.0200 - mean_squared_logarithmic_error: 0.0200
Epoch 194/1000
1095/1095 [==============================] - 0s 86us/step - loss: 0.0197 - mean_squared_logarithmic_error: 0.0197
Epoch 195/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0194 - mean_squared_logarithmic_error: 0.0194
Epoch 196/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0192 - mean_squared_logarithmic_error: 0.0192
Epoch 197/1000
1095/1095 [==============================] - 0s 86us/step - loss: 0.0189 - mean_squared_logarithm

1095/1095 [==============================] - 0s 87us/step - loss: 0.0093 - mean_squared_logarithmic_error: 0.0093
Epoch 254/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0092 - mean_squared_logarithmic_error: 0.0092
Epoch 255/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0091 - mean_squared_logarithmic_error: 0.0091
Epoch 256/1000
1095/1095 [==============================] - 0s 85us/step - loss: 0.0090 - mean_squared_logarithmic_error: 0.0090
Epoch 257/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0089 - mean_squared_logarithmic_error: 0.0089
Epoch 258/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0088 - mean_squared_logarithmic_error: 0.0088
Epoch 259/1000
1095/1095 [==============================] - 0s 84us/step - loss: 0.0087 - mean_squared_logarithmic_error: 0.0087
Epoch 260/1000
1095/1095 [==============================] - 0s 83us/step - loss: 0.0086 - mean_squared_logarithm

...

1095/1095 [==============================] - 0s 88us/step - loss: 5.0497e-04 - mean_squared_logarithmic_error: 5.0497e-04
Epoch 747/1000
1095/1095 [==============================] - 0s 85us/step - loss: 5.0390e-04 - mean_squared_logarithmic_error: 5.0390e-04
Epoch 748/1000
1095/1095 [==============================] - 0s 84us/step - loss: 5.1217e-04 - mean_squared_logarithmic_error: 5.1217e-04
Epoch 749/1000
1095/1095 [==============================] - 0s 84us/step - loss: 4.8785e-04 - mean_squared_logarithmic_error: 4.8785e-04
Epoch 750/1000
1095/1095 [==============================] - 0s 82us/step - loss: 5.1027e-04 - mean_squared_logarithmic_error: 5.1027e-04
Epoch 751/1000
1095/1095 [==============================] - 0s 84us/step - loss: 4.8409e-04 - mean_squared_logarithmic_error: 4.8409e-04
Epoch 752/1000
1095/1095 [==============================] - 0s 84us/step - loss: 4.8559e-04 - mean_squared_logarithmic_error: 4.8559e-04
Epoch 753/1000
1095/1095 [==============================

1095/1095 [==============================] - 0s 85us/step - loss: 3.8328e-04 - mean_squared_logarithmic_error: 3.8328e-04
Epoch 806/1000
1095/1095 [==============================] - 0s 84us/step - loss: 3.7960e-04 - mean_squared_logarithmic_error: 3.7960e-04
Epoch 807/1000
1095/1095 [==============================] - 0s 84us/step - loss: 3.8414e-04 - mean_squared_logarithmic_error: 3.8414e-04
Epoch 808/1000
1095/1095 [==============================] - 0s 84us/step - loss: 3.7926e-04 - mean_squared_logarithmic_error: 3.7926e-04
Epoch 809/1000
1095/1095 [==============================] - 0s 85us/step - loss: 3.8173e-04 - mean_squared_logarithmic_error: 3.8173e-04
Epoch 810/1000
1095/1095 [==============================] - 0s 83us/step - loss: 3.9514e-04 - mean_squared_logarithmic_error: 3.9514e-04
Epoch 811/1000
1095/1095 [==============================] - 0s 85us/step - loss: 3.8910e-04 - mean_squared_logarithmic_error: 3.8910e-04
Epoch 812/1000
1095/1095 [==============================

1095/1095 [==============================] - 0s 84us/step - loss: 3.1187e-04 - mean_squared_logarithmic_error: 3.1187e-04
Epoch 865/1000
1095/1095 [==============================] - 0s 86us/step - loss: 3.0662e-04 - mean_squared_logarithmic_error: 3.0662e-04
Epoch 866/1000
1095/1095 [==============================] - 0s 85us/step - loss: 2.9672e-04 - mean_squared_logarithmic_error: 2.9672e-04
Epoch 867/1000
1095/1095 [==============================] - 0s 83us/step - loss: 2.9203e-04 - mean_squared_logarithmic_error: 2.9203e-04
Epoch 868/1000
1095/1095 [==============================] - 0s 84us/step - loss: 2.9239e-04 - mean_squared_logarithmic_error: 2.9239e-04
Epoch 869/1000
1095/1095 [==============================] - 0s 84us/step - loss: 2.9402e-04 - mean_squared_logarithmic_error: 2.9402e-04
Epoch 870/1000
1095/1095 [==============================] - 0s 85us/step - loss: 3.0581e-04 - mean_squared_logarithmic_error: 3.0581e-04
Epoch 871/1000
1095/1095 [==============================

1095/1095 [==============================] - 0s 88us/step - loss: 2.2888e-04 - mean_squared_logarithmic_error: 2.2888e-04
Epoch 924/1000
1095/1095 [==============================] - 0s 111us/step - loss: 2.2743e-04 - mean_squared_logarithmic_error: 2.2743e-04
Epoch 925/1000
1095/1095 [==============================] - 0s 102us/step - loss: 2.3231e-04 - mean_squared_logarithmic_error: 2.3231e-04
Epoch 926/1000
1095/1095 [==============================] - 0s 89us/step - loss: 2.3073e-04 - mean_squared_logarithmic_error: 2.3073e-04
Epoch 927/1000
1095/1095 [==============================] - 0s 87us/step - loss: 2.3008e-04 - mean_squared_logarithmic_error: 2.3008e-04
Epoch 928/1000
1095/1095 [==============================] - 0s 88us/step - loss: 2.4101e-04 - mean_squared_logarithmic_error: 2.4101e-04
Epoch 929/1000
1095/1095 [==============================] - 0s 91us/step - loss: 2.4292e-04 - mean_squared_logarithmic_error: 2.4292e-04
Epoch 930/1000
1095/1095 [============================

1095/1095 [==============================] - 0s 103us/step - loss: 1.8840e-04 - mean_squared_logarithmic_error: 1.8840e-04
Epoch 983/1000
1095/1095 [==============================] - 0s 107us/step - loss: 1.8014e-04 - mean_squared_logarithmic_error: 1.8014e-04
Epoch 984/1000
1095/1095 [==============================] - 0s 90us/step - loss: 1.8290e-04 - mean_squared_logarithmic_error: 1.8290e-04
Epoch 985/1000
1095/1095 [==============================] - 0s 101us/step - loss: 1.7420e-04 - mean_squared_logarithmic_error: 1.7420e-04
Epoch 986/1000
1095/1095 [==============================] - 0s 101us/step - loss: 1.8498e-04 - mean_squared_logarithmic_error: 1.8498e-04
Epoch 987/1000
1095/1095 [==============================] - 0s 97us/step - loss: 1.8236e-04 - mean_squared_logarithmic_error: 1.8236e-04
Epoch 988/1000
1095/1095 [==============================] - 0s 101us/step - loss: 1.7124e-04 - mean_squared_logarithmic_error: 1.7124e-04
Epoch 989/1000
1095/1095 [=========================

In [ ]:
df_test_orig = pd.read_csv("test.csv")
df_test = df_test_orig.drop(columns=["MiscFeature", "Fence", "PoolQC", "Alley", "Id"])
fill_nulls(df_test, mean, mode)
df_test = encode_categorical_features(df_test, onehot)
df_test = standardize_numerical_features(df_test)
predictions = model.predict(df_test)
predictions = np.squeeze(predictions)
print(predictions.shape)
df_submit = pd.DataFrame({'Id': df_test_orig.Id, 'SalePrice': predictions})
df_submit.to_csv('submission.csv', index=False)